# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

g:\my drive\berkeley_new\diadfit_outer\src\DiadFit\cosmicray_filter.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


'0.0.52'

In [2]:
import os
DayFolder=os.getcwd()
meta_path=DayFolder +  '\MetaData'
spectra_path=DayFolder + '\Spectra'

## Load in specra, and combine

In [3]:
import os.path
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,Unnamed: 0,filename,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,K21-59-FI#1_CRR_DiadFit,103.058244,1286.035116,592.265662,1286.035116,607.841613,0.433741,3.692074,3.098621e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,K21-59-FI#2,103.010689,1286.157463,227.669920,1286.157463,201.585652,0.409447,3.372069,4.144717e-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,K21-59-FI#3_CRR_DiadFit,103.046333,1286.074091,570.408806,1286.074091,582.296621,0.422212,3.767917,3.700715e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,K21-61-FI#1-rep1_CRR_DiadFit,103.310272,1285.431835,65.962068,1285.431835,81.927205,0.577693,3.145730,1.111621e-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,K21-61-FI#1-rep2_CRR_DiadFit,103.304613,1285.435764,607.809709,1285.435764,895.852633,0.568252,3.891746,5.562889e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'FG', 'Cap','window'],
 sort=True)
diad_meta

['K21-59-FI#1-IMG (1).txt',
 'K21-59-FI#1-IMG.txt',
 'K21-59-FI#1.txt',
 'K21-59-FI#2-IMG.txt',
 'K21-59-FI#2.txt',
 'K21-59-FI#3-IMG.txt',
 'K21-59-FI#3.txt',
 'K21-59-IMG5x.txt',
 'K21-60-IMG5x.txt',
 'K21-61-FI#1-IMG.txt',
 'K21-61-FI#1-rep1.txt',
 'K21-61-FI#1-rep2.txt',
 'K21-61-FI#2-IMG.txt',
 'K21-61-FI#2.txt',
 'K21-61-FI#3-IMG.txt',
 'K21-61-FI#3.txt',
 'K21-61-IMG5x.txt',
 'K21-62-FI#1-IMG.txt',
 'K21-62-FI#1-nodiad.txt',
 'K21-62-FI#2-IMG.txt',
 'K21-62-FI#2-nodiad.txt',
 'K21-62-IMG5x.txt',
 'K21-63-FI#1-IMG.txt',
 'K21-63-FI#1.txt',
 'K21-63-IMG5x.txt',
 'K21-64-FI#1-IMG.txt',
 'K21-64-FI#1.txt',
 'K21-64-FI#2-IMG.txt',
 'K21-64-FI#2.txt',
 'K21-64-FI#3-IMG.txt',
 'K21-64-FI#3.txt',
 'K21-64-FI#4-IMG.txt',
 'K21-64-FI#4.txt',
 'K21-64-FI#5-IMG.txt',
 'K21-64-FI#5.txt',
 'K21-64-FI#6-IMG.txt',
 'K21-64-FI#6.txt',
 'K21-64-FI#7-IMG.txt',
 'K21-64-FI#7.txt',
 'K21-64-FI#8-IMG.txt',
 'K21-64-FI#8.txt',
 'K21-64-IMG5x.txt']

## Now get all the important information out of the metadata files

In [6]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=False)
meta['filename'].iloc[0]

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 49.04it/s]

Done


'K21-59-FI#1.txt'

## Then get a simple file name you can stitch with the spectra

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

'K21-59-FI#1'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

In [9]:
file_s

array(['K21-59-FI#1', 'K21-59-FI#2', 'K21-59-FI#3', 'K21-61-FI#1-rep1',
       'K21-61-FI#1-rep2', 'K21-63-FI#1', 'K21-64-FI#2', 'K21-64-FI#8',
       'K21-61-FI#2', 'K21-61-FI#3', 'K21-64-FI#1', 'K21-64-FI#3',
       'K21-64-FI#4', 'K21-64-FI#5', 'K21-64-FI#6', 'K21-64-FI#7'],
      dtype='<U16')

In [26]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
from pickle import load
import pickle
import os
pickle_path=os.getcwd()
file_pkl=pickle_path + '/' + 'Ne_Line_Regression_Example_cali.pck'
with open(file_pkl, 'rb') as file_handle:
    my_loaded_model = pickle.load(file_handle)

In [12]:
df_combo['Ne_Correction']=my_loaded_model(df_combo['sec since midnight'])
df_combo.insert(1,'Corrected_Splitting', df_combo['Ne_Correction']*df_combo['Splitting'])

In [14]:
df_combo.to_excel('FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [17]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None
Sec_Phases.head()

,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb
0,K21-59-FI#1.txt,1151.160629,57.032076,26.539140,NaN,NaN,NaN
1,K21-63-FI#1.txt,1150.591822,81.866472,56.151253,NaN,NaN,NaN
2,K21-64-FI#1.txt,1150.651543,690.347547,437.799964,NaN,NaN,NaN
3,K21-64-FI#2.txt,1150.642811,374.155731,235.883336,NaN,NaN,NaN
4,K21-64-FI#3.txt,1150.614040,1181.443616,738.920572,NaN,NaN,NaN


In [18]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    file_sec_phase

In [24]:
file_sec_phase

array(['K21-59-FI#1', 'K21-63-FI#1', 'K21-64-FI#1', 'K21-64-FI#2',
       'K21-64-FI#3', 'K21-64-FI#5', 'K21-64-FI#6', 'K21-64-FI#7',
       'K21-64-FI#8', 'K21-61-FI#1-rep2'], dtype=object)

In [25]:
df_combo['name_for_matching']

0          K21-59-FI#1.txt
1          K21-59-FI#2.txt
2          K21-59-FI#3.txt
3     K21-61-FI#1-rep1.txt
4     K21-61-FI#1-rep2.txt
5          K21-63-FI#1.txt
6          K21-64-FI#2.txt
7          K21-64-FI#8.txt
8          K21-61-FI#2.txt
9          K21-61-FI#3.txt
10         K21-64-FI#1.txt
11         K21-64-FI#3.txt
12         K21-64-FI#4.txt
13         K21-64-FI#5.txt
14         K21-64-FI#6.txt
15         K21-64-FI#7.txt
Name: name_for_matching, dtype: object

In [30]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo

### Add some more useful columns

In [43]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))

In [44]:
df_combo_sec_phase.to_excel('FI_fitting_w_sec_phases.xlsx')

In [29]:
df_combo_sec_phase

,Unnamed: 0,filename_x,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,...,24hr_time,sec since midnight,Spectral Center,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb
0,0,K21-59-FI#1_CRR_DiadFit,103.058244,1286.035116,592.265662,1286.035116,607.841613,0.433741,3.692074,3.098621e-01,...,12:09:55 PM\n,43795.0,1325.004,K21-59-FI#1.txt,1151.160629,57.032076,26.539140,NaN,NaN,NaN
1,0,K21-59-FI#2,103.010689,1286.157463,227.669920,1286.157463,201.585652,0.409447,3.372069,4.144717e-02,...,12:19:53 PM\n,44393.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,K21-59-FI#3_CRR_DiadFit,103.046333,1286.074091,570.408806,1286.074091,582.296621,0.422212,3.767917,3.700715e-01,...,12:32:26 PM\n,45146.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,K21-61-FI#1-rep1_CRR_DiadFit,103.310272,1285.431835,65.962068,1285.431835,81.927205,0.577693,3.145730,1.111621e-10,...,12:45:44 PM\n,45944.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,K21-61-FI#1-rep2_CRR_DiadFit,103.304613,1285.435764,607.809709,1285.435764,895.852633,0.568252,3.891746,5.562889e-01,...,12:50:47 PM\n,46247.0,1325.004,K21-61-FI#1-rep2.txt,NaN,NaN,NaN,1090.889642,868.243151,43.03457
5,0,K21-63-FI#1_CRR_DiadFit,103.341051,1285.310758,125.549937,1285.310758,210.735722,0.631461,1.840531,6.219952e-01,...,2:18:52 PM\n,51532.0,1325.004,K21-63-FI#1.txt,1150.591822,81.866472,56.151253,NaN,NaN,NaN
6,0,K21-64-FI#2_CRR_DiadFit,103.301001,1285.407637,519.608647,1285.407637,812.182912,0.563376,3.854110,7.243925e-01,...,2:41:12 PM\n,52872.0,1325.004,K21-64-FI#2.txt,1150.642811,374.155731,235.883336,NaN,NaN,NaN
7,0,K21-64-FI#8_CRR_DiadFit,103.288879,1285.392843,158.648198,1285.392843,249.598377,0.619734,1.485895,5.025211e-01,...,5:05:53 PM\n,61553.0,1325.004,K21-64-FI#8.txt,1150.596746,119.215003,74.244442,NaN,NaN,NaN
8,0,K21-61-FI#2,103.300128,1285.470024,988.944858,1285.470074,1433.106847,0.552661,2.918338,5.834877e-01,...,1:08:21 PM\n,47301.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,K21-61-FI#3,103.333811,1285.409241,815.583590,1285.409291,1225.330099,0.556971,2.339883,6.539943e-01,...,1:27:18 PM\n,48438.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
